# Imports

In [1]:
from scheduling_funtions import *
from constraints import *
from data import *
from model import *


# Creating Data

In [2]:
model = create_model()

In [3]:
#Decomp line
# prev_days, prev_staff_works_shift_on_day = read_shifts_from_file(model, "prev_shifts.txt")

In [4]:
staff = create_data(staff_list)
ft_staff = filter_data(staff, ft_only_staff_mask)
midnight_staff = filter_data(staff, midnight_staff_mask)
six_month_new_staff = filter_data(staff, staff_in_first_6_months_mask)
not_ft_staff = list_difference(staff, ft_staff)
not_midnight_staff = list_difference(staff, midnight_staff)

shifts = create_data(shift_list)
midnight_shifts = filter_data(shifts, midnight_shifts_mask)
not_midnight_shifts = list_difference(shifts, midnight_shifts)
late_shifts = filter_data(shifts, late_shifts_mask)
day_shifts = filter_data(shifts, day_shifts_mask)
afternoon_shifts = filter_data(shifts, afternoon_shifts_mask)
ft_shifts = filter_data(shifts, ft_shifts_mask)
not_ft_shifts = list_difference(shifts, ft_shifts)
on_call_shifts = filter_data(shifts, on_call_shifts_mask)
not_on_call_shifts = list_difference(shifts, on_call_shifts)
after_5_shifts = late_shifts + midnight_shifts
after_930_shifts = list_difference(shifts, day_shifts)

days, first_day = create_date_range(1, 1, 2021)
#Decomp line
orig_days = days
# days = prev_days + days
fris = fridays(days, first_day)
weekdays = mondays(days, first_day) + tuesdays(days, first_day) + wednesdays(days, first_day) + thursdays(days, first_day) + fris
sats = saturdays(days, first_day)
suns = sundays(days, first_day)
weekends = sats + suns

# Creating the Model

In [5]:
staff_works_shift_on_day = create_model_variables_long(model, "staff_works_shift_on_day", staff, orig_days, shifts)

#Decomp line
# staff_works_shift_on_day = {**prev_staff_works_shift_on_day, **staff_works_shift_on_day}

staff_works_day = create_model_variables_with_sum(model, "staff_works_day", staff_works_shift_on_day, staff, days, shifts)
staff_doesnt_work_day = not_dict(staff_works_day)
staff_works_afternoon_shift = create_model_variables_with_sum(model, "staff_works_afternoon_shift", staff_works_shift_on_day, staff, days, afternoon_shifts)
staff_doesnt_work_afternoon_shift = not_dict(staff_works_afternoon_shift)
staff_works_midnight_shift = create_model_variables_with_sum(model, "staff_works_midnight_shift", staff_works_shift_on_day, staff, days, midnight_shifts)
staff_works_on_call_shift = create_model_variables_with_sum(model, "staff_works_on_call_shift", staff_works_shift_on_day, staff, days, on_call_shifts)
staff_works_ft_shift = create_model_variables_with_sum(model, "staff_works_ft_shift", staff_works_shift_on_day, staff, days, ft_shifts)
staff_works_late_shift = create_model_variables_with_sum(model, "staff_works_late_shift", staff_works_shift_on_day, staff, days, late_shifts)
staff_doesnt_work_late_shift = not_dict(staff_works_late_shift)
staff_works_after_5_shift = create_model_variables_with_sum(model, "staff_works_after_5_shift", staff_works_shift_on_day, staff, days, after_5_shifts)
staff_doesnt_work_after_5_shift = not_dict(staff_works_after_5_shift)
staff_works_after_930_shift = create_model_variables_with_sum(model, "staff_works_after_930_shift", staff_works_shift_on_day, staff, days, after_930_shifts)
staff_doesnt_work_after_930_shift = not_dict(staff_works_after_930_shift)
staff_works_day_shift = create_model_variables_with_sum(model, "staff_works_day_shift", staff_works_shift_on_day, staff, days, day_shifts)
staff_doesnt_work_day_shift = not_dict(staff_works_day_shift)
staff_productivities = create_staff_variables(model, "staff_productivity", staff_productivity_mask, 0, 6, staff_works_shift_on_day, staff, days, shifts)

obj_int_vars, obj_int_coeffs = empty_minimize_constraints()
obj_bool_vars, obj_bool_coeffs = empty_minimize_constraints()

# Adding Constraints to the Model

In [6]:
max_days_worked_hard_max = 7
max_days_worked_soft_max = 3
max_days_worked_max_cost = MAX

min_days_off_after_midnight_hard_min = 2
min_days_off_after_midnight_soft_min = 4
min_days_off_after_midnight_min_cost = HIGH

max_midnights_in_a_row_hard_max = 2
max_midnights_in_a_row_soft_max = 1
max_midnights_in_a_row_max_cost = MAX

no_late_shift_before_time_off_hard_min = 1
no_late_shift_before_time_off_soft_min = 0
no_late_shift_before_time_off_min_cost = NONE

on_call_rules_before_hard_min = 1
on_call_rules_before_soft_min = 0
on_call_rules_before_min_cost = NONE

on_call_rules_after_hard_min = 1
on_call_rules_after_soft_min = 0
on_call_rules_after_min_cost = NONE

days_off_after_consecutive_shifts_hard_min = 2
days_off_after_consecutive_shifts_soft_min = 3
days_off_after_consecutive_shifts_min_cost = HIGH

days_off_between_late_and_day_shifts_hard_min = 2
days_off_between_late_and_day_shifts_soft_min = 3
days_off_between_late_and_day_shifts_min_cost = HIGH

days_off_between_late_and_afternoon_shifts_hard_min = 2
days_off_between_late_and_afternoon_shifts_soft_min = 3
days_off_between_late_and_afternoon_shifts_min_cost = HIGH

late_shifts_in_a_row_hard_max = 7
late_shifts_in_a_row_soft_max = 3
late_shifts_in_a_row_max_cost = MAX

late_shifts_in_weeks_hard_min = 0
late_shifts_in_weeks_soft_min = 0
late_shifts_in_weeks_min_cost = NONE
late_shifts_in_weeks_hard_max = 14
late_shifts_in_weeks_soft_max = 5
late_shifts_in_weeks_max_cost = MAX

avoid_consecutive_ft_shifts_hard_max = 2
avoid_consecutive_ft_shifts_soft_max = 1
avoid_consecutive_ft_shifts_max_cost = MID

no_nightshifts_before_weekend_off_hard_min = 0
no_nightshifts_before_weekend_off_soft_min = 1
no_nightshifts_before_weekend_off_min_cost = HIGH

minimize_split_weekends_cost = MAX

equalize_weekends_cost = 4
equalize_night_shifts_cost = 4
equalize_late_shifts_cost = 4
equalize_day_shifts_cost = 2
equalize_afternoon_shifts_cost = 2
equalize_weekdays_cost = 1


In [7]:
all_shifts_taken(model,
                staff_works_shift_on_day,
                staff,
                days,
                shifts)

max_days_worked_obj = max_days_worked(model,
                staff_works_day,
                staff,
                days,
                max_days_worked_hard_max,
                max_days_worked_soft_max,
                max_days_worked_max_cost)

obj_bool_vars.extend(max_days_worked_obj[0])
obj_bool_coeffs.extend(max_days_worked_obj[1])

min_days_off_after_midnight_obj = min_days_off_after_midnight(model,
                            staff_doesnt_work_day,
                            staff_works_midnight_shift, 
                            staff, 
                            days, 
                            min_days_off_after_midnight_hard_min, 
                            min_days_off_after_midnight_soft_min, 
                            min_days_off_after_midnight_min_cost)

obj_bool_vars.extend(min_days_off_after_midnight_obj[0])
obj_bool_coeffs.extend(min_days_off_after_midnight_obj[1])

midnight_physicians(model,
                    staff_works_shift_on_day,
                    midnight_staff,
                    days,
                    not_midnight_shifts)

no_midnights_within_six_months(model,
                                staff_works_shift_on_day,
                                six_month_new_staff,
                                days,
                                midnight_shifts)

max_midnights_in_a_row_obj = max_midnights_in_a_row(model,
                        staff_works_midnight_shift,
                        not_midnight_staff,
                        days,
                        max_midnights_in_a_row_hard_max,
                        max_midnights_in_a_row_soft_max,
                        max_midnights_in_a_row_max_cost)

obj_bool_vars.extend(max_midnights_in_a_row_obj[0])
obj_bool_coeffs.extend(max_midnights_in_a_row_obj[1])

ft_physicians(model,
            staff_works_shift_on_day,
            ft_staff,
            days,
            not_ft_shifts)

no_late_shift_before_time_off_obj = no_late_shift_before_time_off(model,
                            staff_works_day,
                            staff_doesnt_work_after_5_shift,
                            staff,
                            no_late_shift_before_time_off_hard_min,
                            no_late_shift_before_time_off_soft_min,
                            no_late_shift_before_time_off_min_cost)

obj_bool_vars.extend(no_late_shift_before_time_off_obj[0])
obj_bool_coeffs.extend(no_late_shift_before_time_off_obj[1])
                            
on_call_rules_before_obj = on_call_rules_before(model,
                    staff_works_on_call_shift,
                    staff_doesnt_work_after_930_shift,
                    staff,
                    days,
                    on_call_rules_before_hard_min,
                    on_call_rules_before_soft_min,
                    on_call_rules_before_min_cost)

obj_bool_vars.extend(on_call_rules_before_obj[0])
obj_bool_coeffs.extend(on_call_rules_before_obj[1])

on_call_rules_after_obj = on_call_rules_after(model,
                    staff_works_on_call_shift,
                    staff_doesnt_work_day_shift,
                    staff,
                    days,
                    on_call_rules_after_hard_min,
                    on_call_rules_after_soft_min,
                    on_call_rules_after_min_cost)

obj_bool_vars.extend(on_call_rules_after_obj[0])
obj_bool_coeffs.extend(on_call_rules_after_obj[1])

days_off_after_consecutive_shifts_obj = days_off_after_consecutive_shifts(model,
                                staff_works_day,
                                staff_works_day,
                                staff,
                                days,
                                days_off_after_consecutive_shifts_hard_min,
                                days_off_after_consecutive_shifts_soft_min,
                                days_off_after_consecutive_shifts_min_cost)

obj_bool_vars.extend(days_off_after_consecutive_shifts_obj[0])
obj_bool_coeffs.extend(days_off_after_consecutive_shifts_obj[1])

In [8]:
transitions_constraints_obj = transitions_constraints(model,
                        staff_works_shift_on_day,
                        staff,
                        days)

obj_bool_vars.extend(transitions_constraints_obj[0])
obj_bool_coeffs.extend(transitions_constraints_obj[1])

days_off_between_late_and_day_shifts_obj = days_off_between_late_and_day_shifts(model,
                                    staff_doesnt_work_day,
                                    staff_works_day_shift,
                                    staff_works_late_shift,
                                    staff,
                                    days,
                                    days_off_between_late_and_day_shifts_hard_min,
                                    days_off_between_late_and_day_shifts_soft_min,
                                    days_off_between_late_and_day_shifts_min_cost)

obj_bool_vars.extend(days_off_between_late_and_day_shifts_obj[0])
obj_bool_coeffs.extend(days_off_between_late_and_day_shifts_obj[1])

days_off_between_late_and_afternoon_shifts_obj = days_off_between_late_and_afternoon_shifts(model,
                                    staff_doesnt_work_day,
                                    staff_works_afternoon_shift,
                                    staff_works_late_shift,
                                    staff,
                                    days,
                                    days_off_between_late_and_afternoon_shifts_hard_min,
                                    days_off_between_late_and_afternoon_shifts_soft_min,
                                    days_off_between_late_and_afternoon_shifts_min_cost)

obj_bool_vars.extend(days_off_between_late_and_afternoon_shifts_obj[0])
obj_bool_coeffs.extend(days_off_between_late_and_afternoon_shifts_obj[1])

late_shifts_in_a_row_obj = late_shifts_in_a_row(model,
                    staff_works_late_shift,
                    staff,
                    days,
                    late_shifts_in_a_row_hard_max,
                    late_shifts_in_a_row_soft_max,
                    late_shifts_in_a_row_max_cost)

obj_bool_vars.extend(late_shifts_in_a_row_obj[0])
obj_bool_coeffs.extend(late_shifts_in_a_row_obj[1])

late_shifts_in_weeks_obj = late_shifts_in_weeks(model,
                        staff_works_late_shift,
                        staff,
                        days,
                        late_shifts_in_weeks_hard_min,
                        late_shifts_in_weeks_soft_min,
                        late_shifts_in_weeks_min_cost,
                        late_shifts_in_weeks_hard_max,
                        late_shifts_in_weeks_soft_max,
                        late_shifts_in_weeks_max_cost)

obj_int_vars.extend(late_shifts_in_weeks_obj[0])
obj_int_coeffs.extend(late_shifts_in_weeks_obj[1])

avoid_consecutive_ft_shifts_obj = avoid_consecutive_ft_shifts(model,
                        staff_works_ft_shift,
                        not_midnight_staff,
                        days,
                        avoid_consecutive_ft_shifts_hard_max,
                        avoid_consecutive_ft_shifts_soft_max,
                        avoid_consecutive_ft_shifts_max_cost)

obj_int_vars.extend(avoid_consecutive_ft_shifts_obj[0])
obj_int_coeffs.extend(avoid_consecutive_ft_shifts_obj[1])

In [9]:
equalize_weekends_obj = equalize_weekends(model,
                    staff_works_day,
                    staff,
                    weekends,
                    equalize_weekends_cost,
                    triangle_costs(len(shifts), len(weekends), len(staff)))

obj_int_vars.extend(equalize_weekends_obj[0])
obj_int_coeffs.extend(equalize_weekends_obj[1])

minimize_split_weekends_obj = minimize_split_weekends(model,
                        staff_works_day,
                        staff,
                        sats,
                        suns,
                        minimize_split_weekends_cost)

obj_int_vars.extend(minimize_split_weekends_obj[0])
obj_int_coeffs.extend(minimize_split_weekends_obj[1])

equalize_night_shifts_obj = equalize_night_shifts(model,
                        staff_works_midnight_shift,
                        staff,
                        days,
                        equalize_night_shifts_cost,
                        triangle_costs(len(midnight_shifts), len(days), len(staff)))

obj_int_vars.extend(equalize_night_shifts_obj[0])
obj_int_coeffs.extend(equalize_night_shifts_obj[1])

no_nightshifts_before_weekend_off_obj = no_nightshifts_before_weekend_off(model,
                                staff_works_day,
                                staff_doesnt_work_after_5_shift,
                                staff,
                                fris,
                                sats,
                                no_nightshifts_before_weekend_off_hard_min,
                                no_nightshifts_before_weekend_off_soft_min,
                                no_nightshifts_before_weekend_off_min_cost)

obj_bool_vars.extend(no_nightshifts_before_weekend_off_obj[0])
obj_bool_coeffs.extend(no_nightshifts_before_weekend_off_obj[1])

equalize_late_shifts_obj = equalize_late_shifts(model,
                    staff_works_late_shift,
                    staff,
                    days,
                    equalize_late_shifts_cost,
                    triangle_costs(len(late_shifts), len(days), len(staff)))

obj_int_vars.extend(equalize_late_shifts_obj[0])
obj_int_coeffs.extend(equalize_late_shifts_obj[1])

equalize_day_shifts_obj = equalize_day_shifts(model,
                    staff_works_day_shift,
                    staff,
                    days,
                    equalize_day_shifts_cost,
                    triangle_costs(len(day_shifts), len(days), len(staff)))

obj_int_vars.extend(equalize_day_shifts_obj[0])
obj_int_coeffs.extend(equalize_day_shifts_obj[1])

equalize_afternoon_shifts_obj = equalize_afternoon_shifts(model,
                            staff_works_afternoon_shift,
                            staff,
                            days,
                            equalize_afternoon_shifts_cost,
                            triangle_costs(len(afternoon_shifts), len(days), len(staff)))

obj_int_vars.extend(equalize_afternoon_shifts_obj[0])
obj_int_coeffs.extend(equalize_afternoon_shifts_obj[1])

equalize_weekdays_obj = equalize_weekdays(model,
                    staff_works_day,
                    staff,
                    weekdays,
                    equalize_weekdays_cost,
                    triangle_costs(len(shifts), len(weekdays), len(staff)))

obj_int_vars.extend(equalize_weekdays_obj[0])
obj_int_coeffs.extend(equalize_weekdays_obj[1])

In [10]:
apply_requests_obj = apply_requests(staff_works_shift_on_day,
                staff_works_day,
                days,
                requests)

obj_int_vars.extend(apply_requests_obj[0])
obj_int_coeffs.extend(apply_requests_obj[1])

apply_productivity_obj = apply_productivity(model,
                    staff_productivities,
                    staff,
                    days,
                    shifts,
                    3)

obj_int_vars.extend(apply_productivity_obj[0])
obj_int_coeffs.extend(apply_productivity_obj[1])

# Solving the Model

In [17]:
# All constraints together takes 1.5 hours to find a feasible solution
import time

start = time.time()

model.Minimize(
        sum(obj_bool_vars[i] * obj_bool_coeffs[i]
            for i in range(len(obj_bool_vars))) +
            sum(obj_int_vars[i] * obj_int_coeffs[i]
            for i in range(len(obj_int_vars))))

solver = create_solver(120)
solution_printer = VarArraySolutionPrinter(staff_works_shift_on_day)
status = solver.Solve(model)

print(solver.StatusName(status))
import collections
results = collections.defaultdict(list)

if status == FEASIBLE or status == OPTIMAL:

    print('Maximum of objective function: %i' % solver.ObjectiveValue())
    print()
    for d in days:
        print('Day', d)
        for s in shifts:
            for m in staff:
                if solver.Value(staff_works_shift_on_day[(m, d, s)]) == 1:
                    results[d].append(staff_list[m])
                    print(staff_list[m], 'works shift', shift_list[s])
        print()

end = time.time()
print(f"time elapsed: {end - start}")

results = [v for k, v in results.items()]

FEASIBLE
Maximum of objective function: 7828

Day 0


IndexError: list index out of range

In [31]:
print(results)

[['Victoria', 'Camila', 'Abigail', 'Isabella', 'Chloe', 'Layla', 'Charlotte', 'Madison', 'Mia', 'Aria', 'Ava', 'Evelyn'], ['Sophia', 'Eleanor', 'Harper', 'Gianna', 'Scarlett', 'Mila', 'Sofia', 'Nora', 'Aria', 'Emma', 'Amelia', 'Luna'], ['Abigail', 'Gianna', 'Emily', 'Olivia', 'Camila', 'Evelyn', 'Penelope', 'Victoria', 'Layla', 'Sophia', 'Ella', 'Elizabeth'], ['Chloe', 'Evelyn', 'Charlotte', 'Harper', 'Emily', 'Elizabeth', 'Isabella', 'Luna', 'Mia', 'Scarlett', 'Sofia', 'Mila'], ['Eleanor', 'Gianna', 'Camila', 'Grace', 'Avery', 'Charlotte', 'Madison', 'Aria', 'Luna', 'Mia', 'Harper', 'Sophia'], ['Olivia', 'Eleanor', 'Isabella', 'Evelyn', 'Nora', 'Elizabeth', 'Chloe', 'Abigail', 'Camila', 'Madison', 'Aria', 'Victoria'], ['Avery', 'Emma', 'Amelia', 'Layla', 'Mila', 'Penelope', 'Victoria', 'Chloe', 'Ella', 'Abigail', 'Luna', 'Mia'], ['Elizabeth', 'Charlotte', 'Layla', 'Ava', 'Scarlett', 'Mila', 'Emma', 'Gianna', 'Evelyn', 'Ella', 'Eleanor', 'Avery'], ['Ava', 'Isabella', 'Charlotte', 'Oliv

In [38]:
# Saving to Excel
import pandas as pd

df = pd.DataFrame(results,
index = days,
columns = shift_list)

df.head()

df.to_csv("output.csv")

# Tests

In [12]:
staff_works_day_results = {}
staff_works_midnight_shift_results = {}
staff_works_late_shift_results = {}
staff_works_day_shift_results = {}
staff_works_shift_on_day_results = {}
staff_works_ft_shift_results = {}
staff_works_after_5_shift_results = {}
staff_works_on_call_shift_results = {}
staff_works_after_930_shift_results = {}
staff_works_afternoon_shift_results = {}
staff_doesnt_work_day_results = {}
staff_doesnt_work_after_930_shift_results = {}
staff_doesnt_work_day_shift_results = {}
staff_doesnt_work_after_5_shift_results = {}
for m in staff:
    staff_works_day_results[m] = [solver.Value(staff_works_day[m, d]) for d in days]
    staff_doesnt_work_day_results[m] = [solver.Value(staff_doesnt_work_day[m, d]) for d in days]
    staff_works_midnight_shift_results[m] = [solver.Value(staff_works_midnight_shift[m, d]) for d in days]
    staff_works_late_shift_results[m] = [solver.Value(staff_works_late_shift[m, d]) for d in days]
    staff_works_day_shift_results[m] = [solver.Value(staff_works_day_shift[m, d]) for d in days]
    staff_works_ft_shift_results[m] = [solver.Value(staff_works_ft_shift[m, d]) for d in days]
    staff_works_after_5_shift_results[m] = [solver.Value(staff_works_after_5_shift[m,d]) for d in days]
    staff_works_on_call_shift_results[m] = [solver.Value(staff_works_on_call_shift[m,d]) for d in days]
    staff_works_after_930_shift_results[m] = [solver.Value(staff_works_after_930_shift[m,d]) for d in days]
    staff_works_afternoon_shift_results[m] = [solver.Value(staff_works_afternoon_shift[m,d]) for d in days]
    staff_doesnt_work_after_930_shift_results[m] = [solver.Value(staff_doesnt_work_after_930_shift[m,d]) for d in days]
    staff_doesnt_work_day_shift_results[m] = [solver.Value(staff_doesnt_work_day_shift[m,d]) for d in days]
    staff_doesnt_work_after_5_shift_results[m] = [solver.Value(staff_doesnt_work_after_5_shift[m,d]) for d in days]
    for d in days:
        staff_works_shift_on_day_results[m, d] = [solver.Value(staff_works_shift_on_day[(m, d, s)]) for s in shifts]

## Hard Constraints

In [13]:
from test import *

all_shifts_taken_test(staff_works_shift_on_day_results, staff, days, shifts)
  
no_two_shifts_on_same_day_test(staff_works_shift_on_day_results)

max_days_worked_test(staff_works_day_results,
                    max_days_worked_hard_max,
                    max_days_worked_soft_max, 
                    obj_result(solver, max_days_worked_obj))

min_days_off_after_midnight_test(staff_doesnt_work_day_results,
                                 staff_works_midnight_shift_results,
                                 min_days_off_after_midnight_hard_min, 
                                 min_days_off_after_midnight_soft_min,
                                 obj_result(solver, min_days_off_after_midnight_obj))

midnight_physicians_test(staff_works_shift_on_day_results, shifts)

no_midnights_within_six_months_test(staff_works_midnight_shift_results, shifts)

max_midnights_in_a_row_test(staff_works_midnight_shift_results, 
                            max_midnights_in_a_row_hard_max,
                            max_midnights_in_a_row_soft_max,
                            obj_result(solver, max_midnights_in_a_row_obj))

ft_physicians_test(staff_works_shift_on_day_results, shifts)

no_late_shift_before_time_off_test(staff_works_day_results, staff_works_after_5_shift_results)

on_call_rules_before_test(staff_doesnt_work_after_930_shift_results,
                            staff_works_on_call_shift_results,
                            on_call_rules_before_hard_min,
                            on_call_rules_before_soft_min, 
                            obj_result(solver, on_call_rules_before_obj))

on_call_rules_after_test(staff_doesnt_work_day_shift_results, 
                        staff_works_on_call_shift_results, 
                        on_call_rules_after_hard_min, 
                        on_call_rules_after_soft_min, 
                        obj_result(solver, on_call_rules_after_obj))

days_off_after_consecutive_shifts_test(staff_doesnt_work_day_results, 
                                        staff_works_day_results, 
                                        days_off_after_consecutive_shifts_hard_min, 
                                        days_off_after_consecutive_shifts_soft_min, 
                                        obj_result(solver, days_off_after_consecutive_shifts_obj))

print("ALL HARD CONSTRAINTS SATISFIED")

Max days worked consecutively
Objective function: 900
	3 from range 3 to 7: 9

	4 from range 3 to 7: 7

	5 from range 3 to 7: 4

	6 from range 3 to 7: 3

	7 from range 3 to 7: 1


Min days off after midnight shift
Objective function: 600
	2 from range 2 to 4: 0

	3 from range 2 to 4: 15

	4 from range 2 to 4: 3


Max midnights worked consecutively
Objective function: 120
	1 from range 1 to 2: 29

	2 from range 1 to 2: 2


No shifts after 930 before on call
Objective function: 0

No early morning shifts post on call
Objective function: 0

days_off_after_consecutive_shifts
Objective function: 0
	2 from range 2 to 3: 0

	3 from range 2 to 3: 0


ALL HARD CONSTRAINTS SATISFIED


## Pie in the sky

In [14]:
#Ensure requests work as intended
number_of_requests_respected = 0
number_of_requests_disrespected = 0

for staff2, day2, shift2, coef in requests:
    if day2 == -1:
        continue
    if shift2 == -1 and coef > 0:
        if solver.Value(staff_works_day_results[staff2][day2]):
            number_of_requests_disrespected += 1
        else:
            number_of_requests_respected += 1
    elif shift2 == -1 and coef < 0:
        if solver.Value(staff_works_day_results[staff2][day2]):
            number_of_requests_respected += 1
        else:
            number_of_requests_disrespected += 1
    elif coef > 0:
        if solver.Value(staff_works_shift_on_day_results[staff2, day2][shift2]):
            number_of_requests_disrespected += 1
        else:
            number_of_requests_respected += 1
    elif coef < 0:
        if solver.Value(staff_works_shift_on_day_results[staff2, day2][shift2]):
            number_of_requests_respected += 1
        else:
            number_of_requests_disrespected += 1

print(f"number of requests respected: {number_of_requests_respected}")
print(f"number of requests disrespected: {number_of_requests_disrespected}")

number of requests respected: 0
number of requests disrespected: 0


In [15]:
#Ensure requests work as intended
productivity_under = 0
productivity_over = 0

for d in days:
    for wind in window(shifts[:-1], 3):
        prod = [solver.Value(staff_productivities[m,d,s])
        for m in staff 
        for s in wind]
        sums = sum(prod)
        if sums < 6:
            productivity_under += 1
        if sums > 15:
            productivity_over += 1

print(f"number of times theres too little productivity: {productivity_under}")
print(f"number of times theres too much productivity: {productivity_over}")

number of times theres too little productivity: 6
number of times theres too much productivity: 0


## Soft Constraints

In [16]:
transition_constraint_test(staff_works_shift_on_day_results, days)

days_off_between_late_and_day_shifts_test(staff_doesnt_work_day_results,
                                            staff_works_late_shift_results,
                                            staff_works_day_shift_results, 
                                            days_off_between_late_and_day_shifts_hard_min, 
                                            days_off_between_late_and_day_shifts_soft_min,
                                            obj_result(solver, days_off_between_late_and_day_shifts_obj))

days_off_between_late_and_afternoon_shifts_test(staff_doesnt_work_day_results,
                                                staff_works_late_shift_results, 
                                                staff_works_afternoon_shift_results, 
                                                days_off_between_late_and_afternoon_shifts_hard_min, 
                                                days_off_between_late_and_afternoon_shifts_soft_min, 
                                                obj_result(solver, days_off_between_late_and_afternoon_shifts_obj))

late_shifts_in_a_row_test(staff_works_late_shift_results,
                         late_shifts_in_a_row_hard_max, 
                         late_shifts_in_a_row_soft_max, 
                         obj_result(solver, late_shifts_in_a_row_obj))

avoid_consecutive_ft_shifts_test(staff_works_ft_shift_results,
                                 avoid_consecutive_ft_shifts_hard_max, 
                                 avoid_consecutive_ft_shifts_soft_max,
                                 obj_result(solver, avoid_consecutive_ft_shifts_obj))

Number of times shifts jump forwards 2.5 hours: 34
Number of times shifts jump backwards 1.5 hours: 34

days_off_between_late_and_day_shifts
Objective function: 0
	2 from range 2 to 3: 0

	3 from range 2 to 3: 0


days_off_between_late_and_afternoon_shifts
Objective function: 0
	2 from range 2 to 3: 0

	3 from range 2 to 3: 0


days_off_between_late_and_afternoon_shifts
Objective function: 0
	3 from range 3 to 7: 1

	4 from range 3 to 7: 0

	5 from range 3 to 7: 0

	6 from range 3 to 7: 0

	7 from range 3 to 7: 0


days_off_between_late_and_afternoon_shifts
Objective function: 144
	1 from range 1 to 2: 56

	2 from range 1 to 2: 6




## Goals

In [17]:
weekends_worked = {}
weekdays_worked = {}
first = {}
second = {}
for m in staff:
    weekends_worked[m] = [solver.Value(staff_works_day[m, d]) for d in weekends]
    weekdays_worked[m] = [solver.Value(staff_works_day[m, d]) for d in weekdays]
    first[m] = [solver.Value(staff_doesnt_work_after_5_shift[m, d])
                    for d in fris] + [model.NewConstant(1)] * no_nightshifts_before_weekend_off_hard_min
    second[m] = [model.NewConstant(1)] * no_nightshifts_before_weekend_off_hard_min + [solver.Value(staff_works_day[m, d]) for d in sats]

In [18]:
equalize_weekends_test(weekends_worked, triangle_costs(len(shifts), len(weekends), len(staff)), obj_result(solver, equalize_weekends_obj))
equalize_night_shifts_test(staff_works_midnight_shift_results, triangle_costs(len(midnight_shifts), len(days), len(staff)), obj_result(solver, equalize_night_shifts_obj))
equalize_late_shifts_test(staff_works_late_shift_results, triangle_costs(len(late_shifts), len(days), len(staff)), obj_result(solver, equalize_late_shifts_obj))
equalize_day_shifts_test(staff_works_day_shift_results, triangle_costs(len(day_shifts), len(days), len(staff)), obj_result(solver, equalize_day_shifts_obj))
equalize_afternoon_shifts_test(staff_works_afternoon_shift_results, triangle_costs(len(afternoon_shifts), len(days), len(staff)), obj_result(solver, equalize_afternoon_shifts_obj))
equalize_weekdays_test(weekdays_worked, triangle_costs(len(shifts), len(weekdays), len(staff)), obj_result(solver, equalize_weekdays_obj))

minimize_split_weekends_test(solver, staff_works_day, staff, sats, suns)

no_nightshifts_before_weekend_off_test(first, 
                                        second, 
                                        no_nightshifts_before_weekend_off_hard_min, 
                                        no_nightshifts_before_weekend_off_soft_min, 
                                        obj_result(solver, no_nightshifts_before_weekend_off_obj))

Weekend Equalization
Objective function: 0
Difference from target of 4: 0
night shift Equalization
Objective function: 144
Difference from target of 2: 29
night shift Equalization
Objective function: 164
Difference from target of 4: 27
night shift Equalization
Objective function: 144
Difference from target of 4: 27
night shift Equalization
Objective function: 164
Difference from target of 5: 26
Weekend Equalization
Objective function: 120
Difference from target of 9: 18
[0, 1, 0, 1, 0]
[1, 0, 1, 0, 0]
1
8
15
22
29
[1, 0, 1, 0, 1]
[0, 1, 0, 0, 0]
1
8
15
22
29
[0, 1, 1, 1, 0]
[0, 0, 0, 1, 0]
1
8
15
22
29
[0, 1, 0, 1, 1]
[0, 0, 0, 1, 0]
1
8
15
22
29
[1, 0, 0, 0, 0]
[1, 1, 1, 0, 0]
1
8
15
22
29
[1, 0, 0, 0, 0]
[0, 1, 1, 1, 0]
1
8
15
22
29
[0, 1, 0, 0, 0]
[0, 1, 0, 1, 1]
1
8
15
22
29
[0, 0, 1, 1, 0]
[0, 0, 1, 1, 0]
1
8
15
22
29
[0, 0, 0, 0, 0]
[1, 0, 1, 1, 1]
1
8
15
22
29
[1, 1, 0, 0, 1]
[0, 1, 0, 0, 0]
1
8
15
22
29
[0, 0, 1, 0, 1]
[1, 0, 0, 0, 1]
1
8
15
22
29
[1, 1, 0, 0, 0]
[1, 0, 0, 1, 0

In [19]:
new_works = {k:v for (k,v) in staff_works_shift_on_day.items() if k[1] >= 0}
save_shifts_to_file("prev_shifts.txt2", new_works, len(staff_list), len(days), len(shift_list), solver)

In [20]:
prev_days, prev_shifts = read_shifts_from_file(model, "prev_shifts.txt")
for d in prev_days:
    # print('Day', d)
    for s in shifts:
        for m in staff:
            if solver.Value(prev_shifts[(m, d, s)]) == 1:
                print(staff_list[m], 'works shift', shift_list[s])
    print()

Emily works shift 0700 - 1500
Camila works shift 0730 - 1530 (FT)
Abigail works shift 0930 - 1730
Isabella works shift 1200 - 2000
Chloe works shift 1400 - 2200
Layla works shift 1530 - 2330 (FT)
Charlotte works shift 1600 - 2400
Madison works shift 1800 - 0200
Mia works shift 2000 - 0400
Aria works shift 2200 - 0400
Ava works shift 2359 - 0700
Evelyn works shift On Call

Sophia works shift 0700 - 1500
Eleanor works shift 0730 - 1530 (FT)
Harper works shift 0930 - 1730
Gianna works shift 1200 - 2000
Scarlett works shift 1400 - 2200
Mila works shift 1530 - 2330 (FT)
Sofia works shift 1600 - 2400
Nora works shift 1800 - 0200
Aria works shift 2000 - 0400
Emma works shift 2200 - 0400
Amelia works shift 2359 - 0700
Luna works shift On Call

Abigail works shift 0700 - 1500
Gianna works shift 0730 - 1530 (FT)
Emily works shift 0930 - 1730
Olivia works shift 1200 - 2000
Camila works shift 1400 - 2200
Evelyn works shift 1530 - 2330 (FT)
Penelope works shift 1600 - 2400
Victoria works shift 1800